In [22]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

In [23]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")

uv_train, uv_test = train_test_split(user_vectors, test_size=0.2, random_state=8)

ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")
anime = pd.read_csv(r"D:\dataset\anime_cleaned.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [45]:
encoder = LabelEncoder()
t_encoder = LabelEncoder()
final_vectors = uv_train.copy()
test_vectors = uv_test.copy()
x = encoder.fit(final_vectors["username"])
t = t_encoder.fit(test_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
test_enc = t.transform(test_vectors["username"])

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc
test_vectors["username"] = test_enc
#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
test_arrays = test_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])
#final_vectors
uv_train
#test_arrays
#test_vectors
#uv_test

,username,experience,gender,generation,fav_anime_period,Thriller,Ecchi,of,Arts,Military,Horror,Super,Kids,Drama,Police,Adventure,Vampire,Comedy,Life,Action,Psychological,Shoujo,Romance,Magic,Supernatural,Parody,Samurai,Game,Demons,Sci-Fi,Martial,Seinen,Shounen,School,Historical,Ai,Harem,Music,Space,Sports,Slice,Mecha,Fantasy,Power,Mystery,Dementia
7053,nemeklau,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4363,TheoriginalMadD,3,1,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8451,Napalmitos,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1478,MrYosuko,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8047,GenkaiShinigami,2,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5480,NirvashThurston,2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
6995,Harmleikur,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2181,Xanquis,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2033,Ali_Mohamd,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Compute K-Nearest Neighbours

In [49]:
#function that returns a list of indexes of most similar users in "final_arrays" array
def neighbours(user):
    #user = np.array([7537, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    #maybe add some weightings
    train = nc.fit(final_arrays) #fit the model with the user vector data (training)
    n = train.kneighbors([user], return_distance = False)
    return n

#returns a dataframe of users that were in k-nearest neighbours list 
def sim_frame(neighbours):
    sim_names = []
    #neighbours = neighbours()

    for i in neighbours[0]: #loop through nested array
        #print(final_arrays[i][0])
        name = x.inverse_transform([final_arrays[i][0]]) #find usernames of similar users
        sim_names.append(name)

    #build new dataframe of neighbours
    neighbour_frame = pd.DataFrame()
    for i in users.columns:
        neighbour_frame[i] = None

    for name in sim_names:
        user_row = users.loc[users["username"]==name[0]]
        neighbour_frame = pd.concat([neighbour_frame, user_row])

    pd.options.display.max_rows = 999
    pd.set_option('display.max_columns', 500)
    return neighbour_frame

neighbour_ids = neighbours(np.array([6490, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0]))
#print(neighbour_ids)
neighbour_frame = sim_frame(neighbour_ids)
neighbour_frame
#final_vectors.loc[final_vectors["username"]==7537]
#(final_vectors.columns)    

,user_id,username,user_completed,stats_mean_score,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
3550,162146,yare_yare,294,0.00,Active,Female,1995,5560,Gen Z,"['Action', 'Supernatural', 'Comedy']",New-Gen
3008,5532574,xsimonax,42,8.74,Active,Female,2000,3226,Gen Z,"['Action', 'Comedy', 'Shounen']",New-Gen
9238,4978325,xXSpicyNekoXx,137,8.11,Active,Female,1997,4423,Gen Z,"['Action', 'Comedy', 'Seinen']",New-Gen
2641,3343903,xScarletDreams,308,8.51,Active,Female,1995,5201,Gen Z,"['Action', 'Fantasy', 'Shounen']",New-Gen
6152,4162633,uilbp,181,8.89,Active,Female,1995,3844,Gen Z,"['Action', 'Shounen', 'Supernatural']",New-Gen


Create anime recommendations

In [26]:

#Put anime that have most members at the top of each users "lists"
def recommendations(neighbours, media_type):
    #Get top rated TV shows or movies from each user
    suggestions = [] #anime_id's of recommended anime
    for i in neighbours["username"]:
        user_ratings = ratings.loc[ratings["username"]==i]
        tv_shows = user_ratings[user_ratings["type"]=="TV"]
        movies = user_ratings[user_ratings["type"]=="Movie"]

        if media_type == "TV":
            show_ratings = tv_shows
        else:
            show_ratings = movies
        
        #There may be 0 movies in the user's top 10 list in which case we skip that user's recommendationan
        if len(show_ratings) > 1:
            top_rated = show_ratings.iloc[0]["title"] #change to ids when creating the website to lookup the anime
            suggestions.append(top_rated)
            #print(top_rated)

    if len(suggestions)==0:
        print("No movies to recommend :/")
    
    return suggestions

recommendations(neighbour_frame, "TV")
#ratings

['No.6',
 'Tokyo Ghoul',
 'Bungou Stray Dogs 2nd Season',
 'Nabari no Ou',
 'Boku no Hero Academia']

Based on model fit with training data (K-Neighbours), evaluate recommended anime for users in test set


In [27]:
real_ratings = pd.read_csv(r"D:\dataset\animelists_cleaned.csv")

In [28]:
def evaluate_users():
    #Create kneighbours dataframe for all users in the dataset
    overall_scores = []
    num_likes = []
    num_shows_watched = []
    for user in test_arrays:
        user = np.array(user)
        #print([user[0]])
        username = t.inverse_transform([user[0]])[0]
        #print(username)
        user_neighbours = neighbours(user)
        nframe = sim_frame(user_neighbours)
        #print(user_neighbours)
        user_recommendations = recommendations(nframe, "TV")
        #print(nframe["username"], user_recommendations)
        
        #Use original ratings dataframe with all shows user has rated instead of simply the top 10
        rt = real_ratings.loc[real_ratings["username"]==username]
        rt = rt.loc[rt["my_watched_episodes"]>3]
        score = 0
        likes = 0
        shows_watched = 0
        #Check whether recommended shows are present in original ratings table
        for show in user_recommendations:
            a_id = int(anime.loc[anime["title"]==show]["anime_id"])
            #u_ratings = real_ratings.loc[real_ratings["username"]==username]
            if int(a_id) in list(rt["anime_id"]): #if user watched the recommended show, give 1 point
                rating_row = real_ratings.loc[(real_ratings["username"]==username)&(real_ratings["anime_id"]==a_id)]
                #print(rating_row)
                shows_watched+=1
                score+=1 #add a point to overall score because user has watched recommended show before
                if int(rating_row["my_score"])>=7:#user "likes" the show if they rated it higher than or equal to 7
                    score+=1 #add an extra point to overall score if the user liked the show
                    likes+=1

        score = (score/10)*100 #max number of points is 2 x number of neighbours (k): 10
        num_likes.append(likes)
        overall_scores.append(score)
        num_shows_watched.append(shows_watched)
        #print(f"Score for user {username}: {score}%")

        #remove percentage 
        #standard deviation of results
        #distribution of scores - how many score are 1,2,3,4,5,6,7,8,9,10
        #plot on graph
        #potentially take top 2 instead of top 1

    avg_score = np.average(overall_scores)
    avg_likes = np.average(num_likes)
    avg_shows_watched = np.average(num_shows_watched)
    print(f"Average overall recommendation score for Test users: {avg_score}%")
    print(f"Average number of recommended shows that users have watched before: {avg_shows_watched}")
    print(f"Average number of recommended anime that users liked: {avg_likes}")
    return avg_score, avg_shows_watched, avg_likes

#evaluate_users() #147minutes
#include analysis of distance metric between neighbours e.g. cosine, euclidean 

Collaborative filtering for additional suggestions

In [29]:
rating_vectors = pd.read_csv(r"D:\dataset\encoding\collab_scores.csv")

In [102]:
def r_neighbours(anime):
    nc = NearestNeighbors(n_neighbors = 5, metric="cosine") #euclidean
    train = nc.fit(rating_vectors.values) #fit the model with the ratings data
    n = train.kneighbors([anime], return_distance = False)
    return n
print(len(rating_vectors))

3149


In [87]:

def r_neighbour_ids():
    sample_anime = list(rating_vectors.loc[rating_vectors["anime_id"]==11013].values)
    n_indexes = r_neighbours(sample_anime[0])[0]
    print(n_indexes)
    n_ids = []
    for i in n_indexes:
        anime_id = rating_vectors.iloc[i]["anime_id"]
        n_ids.append(anime_id)
    print(n_ids)
    return n_ids
#r_neighbour_ids()   

def get_additional_anime():
    more_recs = pd.DataFrame()
    for i in r_neighbour_ids():
        anime_row = anime.loc[anime["anime_id"]==i]
        #print(anime_row)
        more_recs = pd.concat([more_recs, anime_row])
        #more_recs = more_recs.loc[:,["anime_id","title","studio", "genre","time_period", "fame"]]
    return more_recs
get_additional_anime()

[   0   42  811 1450 1773]
[11013, 14713, 23289, 25013, 12967]


,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",24.0,2012.0
45,14713,Kamisama Hajimemashita,Kamisama Kiss,神様はじめました,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,13,Finished Airing,False,"Oct 2, 2012 to Dec 25, 2012","{'from': '2012-10-02', 'to': '2012-12-25'}",24 min. per ep.,PG-13 - Teens 13 or older,8.14,138954,383.0,259,266342,6387,NaN,Fall 2012,Tuesdays at 02:05 (JST),"{'Adaptation': [{'mal_id': 8157, 'type': 'mang...","TV Tokyo, Dentsu, Sotsu, Pony Canyon, Hakusens...",Funimation,TMS Entertainment,"Comedy, Demons, Supernatural, Romance, Fantasy...","['""Kamisama Hajimemashita (神様はじめました)"" by Hanae']","['#1: ""Kamisama Onegai (神様お願い)"" by Hanae (eps ...",24.0,2012.0
1568,23289,Gekkan Shoujo Nozaki-kun,Monthly Girls&#039; Nozaki-kun,月刊少女野崎くん,Gekkan Shoujo Nozaki-kun,https://myanimelist.cdn-dena.com/images/anime/...,TV,4-koma manga,12,Finished Airing,False,"Jul 7, 2014 to Sep 22, 2014","{'from': '2014-07-07', 'to': '2014-09-22'}",24 min. per ep.,PG-13 - Teens 13 or older,8.15,230443,372.0,109,419727,7460,The voice acting cast for the anime differs fr...,Summer 2014,Mondays at 01:05 (JST),"{'Adaptation': [{'mal_id': 29211, 'type': 'man...","TV Tokyo, Frontier Works, Media Factory, AT-X,...",Sentai Filmworks,Doga Kobo,"Comedy, Romance, School","['""Kimi ja Nakya Dame Mitai (君じゃなきゃダメみたい)"" by ...","['""Ura Omote Fortune (ウラオモテ・フォーチュン)"" by Chiyo ...",24.0,2014.0
2977,25013,Akatsuki no Yona,Yona of the Dawn,暁のヨナ,Yona: The girl standing in the blush of dawn,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,24,Finished Airing,False,"Oct 7, 2014 to Mar 24, 2015","{'from': '2014-10-07', 'to': '2015-03-24'}",24 min. per ep.,PG-13 - Teens 13 or older,8.19,166612,320.0,165,339298,8088,Episodes 1-2 were previewed at a screening at ...,Fall 2014,Tuesdays at 23:00 (JST),"{'Adaptation': [{'mal_id': 21525, 'type': 'man...","VAP, Hakusensha, AT-X, Delfi Sound, Marvelous ...",Funimation,Studio Pierrot,"Action, Adventure, Comedy, Fantasy, Romance, S...","['#1: ""Akatsuki no Yona (暁のヨナ)"" by Kunihiko Ry...","['#1: ""Yoru (夜)"" by vistlip (eps 1-14)', '#2: ...",24.0,2014.0
3634,12967,Arcana Famiglia,La storia della Arcana Famiglia,アルカナ・ファミリア －La storia della Arcana Famiglia－,Arcana Famiglia: La Storia Della Arcana Famiglia,https://myanimelist.cdn-dena.com/images/anime/...,TV,Visual novel,12,Finished Airing,False,"Jul 1, 2012 to Sep 16, 2012","{'from': '2012-07-01', 'to': '2012-09-16'}",24 min. per ep.,PG-13 - Teens 13 or older,6.41,63122,5949.0,635,132172,275,The first episode received an early preview sc...,Summer 2012,Unknown,"{'Adaptation': [{'mal_id': 35507, 'type': 'man...","Frontier Works, Movic, Warner Bros., Showgate",Sentai Filmworks,J.C.Staff,"Action, Harem, Supernatural, Romance, Shoujo","['""Magenta Another Sky"" by Hitomi Harada']","['""Pieces of Treasure"" by Liberta & Nova (CV.J...",24.0,2012.0


In [ ]:

#def evaluate_ratings():
    

Function that converts user information from website into vector